# Some Typical Financial Calculations

In [1]:
using StatsBase, Roots, SpecialFunctions 

include("printmat.jl")            #function for prettier matrix printing

printlnPs (generic function with 1 method)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
    default(show=false)               #for pyplot: avoids pop-ups
else    
    gr(size=(600,400))
    default(show=true)
end

# OLS and Autocorrelations

### Load Data

In [3]:
xx  = readdlm("Data/MyData.csv",',',header=true)
x   = xx[1]
ym  = x[:,1]                 #yearmonth
Rme = x[:,2]                 #market excess return
Rf  = x[:,3]                 #interest rate
R   = x[:,4]                 #return small growth stocks
Re  = R - Rf                 #excess returns
T   = size(Rme,1)

388

### CAPM regression and Autocorrelations

In [4]:
x    = [ones(T) Rme]             #regressors
y    = copy(Re)                  #to get standard OLS notation
b    = x\y                       #OLS
u    = y - x*b                   #residuals
covb = inv(x'x)*var(u)           #cov(b)
stdb = sqrt(diag(covb))          #std(b)
R2a  = 1 - var(u)/var(y)

println("OLS coefficients (col 1) and their std (col 2)")
printmat([b stdb])
printlnPs("R2: ",R2a)
printlnPs("no. of observations: ",T)

OLS coefficients (col 1) and their std (col 2)
    -0.504     0.304
     1.341     0.066

R2:      0.519
no. of observations:        388


In [5]:
plags = 1:5
xCorr = autocor(Re,plags)

println("\n lag (col 1) autocorrr (col 2) and its t-stat (col 3)")
printmat([plags xCorr sqrt(T)*xCorr])


 lag (col 1) autocorrr (col 2) and its t-stat (col 3)
     1.000     0.216     4.253
     2.000     0.002     0.046
     3.000    -0.018    -0.359
     4.000    -0.065    -1.289
     5.000    -0.027    -0.536



# Value at Risk

In [6]:
lambda = 0.95                         #weight on old volatility

Rm     = Rme + Rf                     #equity market return in %
sigma2 = var(Rm)
mu     = mean(Rm)

vol2 = fill(sigma2,T)                 #RiskMetrics approach to estimate variance     
for t = 2:T
  vol2[t] = lambda*vol2[t-1] + (1-lambda)*(Rm[t-1]-mu)^2    
end

VaR95 = -(mu - 1.64*sqrt.(vol2))      #VaR at 95% level

388-element Array{Float64,1}:
 6.4961 
 6.44361
 6.38066
 6.47026
 6.28037
 6.14912
 6.09348
 5.91481
 6.02466
 5.85366
 6.33799
 6.41068
 6.25113
 ⋮      
 8.32903
 8.67378
 8.70052
 8.71098
 8.67052
 8.89147
 8.69606
 8.45105
 8.44683
 8.21349
 8.03808
 7.81245

In [7]:
YearFrac = floor(ym/100) + (mod(ym,100)-1)/12    #eg 1990.92 for Dec 1990

plot1 = plot(YearFrac,VaR95,color=:blue,legend=false)
plot!(xlims=(1978,2012),ylims=(0,11))
title!("1-month Value at Risk (95%) for US equity market")
ylabel!("%")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 1-month Value at Risk (95%) for US equity market 
 
 
 % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 53.4358,171.425 54.8061,173.15 56.1764,175.219 57.5466,172.274 58.9169,178.516 60.2872,182.83 61.6574,184.659 63.0277,190.532 64.398,186.921 65.7682,192.542 
 67.1385,176.622 68.5088,174.233 69.8791,179.477 71.2493,176.268 72.6196,182.148 73.9899,139.604 75.3601,142.05 76.7304,142.513 78.1007,147.523 79.471,145.028 
 80.8412,151.5 82.2115,157.268 83.5818,163.635 84.952,145.263 86.3223,146.788 87.6926,146.791 89.0628,153.619 90.4331,156.778 91.8034,162.086 93.1737,168.568 
 94.5439,173.652 95.9142,179.316 97.2845,171.944 98.6547,162.134 100.025,161.561 101.395,164.492 102.766,166.636 104.136,169.104 105.506,164.406 106.876,169.629 
 108.247,172.743 109.617,174.514 110.987,177.147 112.357,180.35 113.728,152.138 115.098,158.787 116.468,133.852 117.839,136.621 119.209,143.733 120.579,148.042 
 121.949,153.836 123.32,158.907 124.69,154.109 126.06,160.8 127.43,165.192 128.801,166.495 130.171,172.753 131.541,178.897 132.912,180.446 134.282,185.347 
 135.652,189.878 137.022,193.909 138.393,191.644 139.763,197.377 141.133,202.784 142.503,195.124 143.874,200.239 145.244,202.532 146.614,174.673 147.984,180.559 
 149.355,186.257 150.725,190.732 152.095,195.944 153.466,183.441 154.836,189.279 156.206,194.625 157.576,199.783 158.947,198.458 160.317,203.898 161.687,208.952 
 163.057,213.456 164.428,209.727 165.798,210.966 167.168,204.232 168.539,206.007 169.909,211.405 171.279,203.29 172.649,202.332 174.02,206.673 175.39,206.416 
 176.76,211.755 178.13,200.051 179.501,195.272 180.871,175.777 182.241,177.36 183.612,183.416 184.982,185.14 186.352,150.316 187.722,153.261 189.093,159.503 
 190.463,163.929 191.833,170.311 193.203,173.321 194.574,176.115 195.944,180.099 197.314,183.134 198.685,53.4956 200.055,49.0711 201.425,51.2962 202.795,58.213 
 204.166,63.8546 205.536,71.3241 206.906,80.1325 208.276,88.4976 209.647,93.0922 211.017,100.633 212.387,105.264 213.757,111.517 215.128,119.184 216.498,124.998 
 217.868,132.16 219.239,131.494 220.609,136.921 221.979,143.746 223.349,146.888 224.72,151.512 226.09,157.638 227.46,151.895 228.83,158.284 230.201,164.486 
 231.571,166.36 232.941,172.603 234.312,178.66 235.682,165.454 237.052,171.821 238.422,177.523 239.793,179.062 241.163,166.754 242.533,172.516 243.903,177.578 
 245.274,153.354 246.644,148.769 248.014,154.172 249.385,152.431 250.755,158.172 252.125,160.547 253.495,155.07 254.866,160.798 256.236,167.195 257.606,170.881 
 258.976,168.436 260.347,170.885 261.717,176.37 263.087,181.114 264.457,186.974 265.828,185.658 267.198,163.647 268.568,169.724 269.939,176.015 271.309,178.631 
 272.679,184.695 274.049,190.582 275.42,193.509 276.79,196.375 278.16,198.78 279.53,204.349 280.901,209.788 282.271,211.961 283.641,217.014 285.012,222.116 
 286.382,227.012 287.752,231.046 289.122,230.795 290.493,234.118 291.863,238.703 293.233,242.755 294.603,243.689 295.974,247.725 297.344,251.811 298.714,252.59 
 300.085,256.441 301.455,258.543 302.825,257.145 304.195,247.324 305.566,251.673 306.936,255.911 308.306,253.477 309.676,255.854 311.047,255.337 312.417,255.751 
 313.787,259.815 315.158,253.584 316.528,257.757 317.898,261.387 319.268,261.416 320.639,264.12 322.009,267.031 323.379,268.187 324.749,269.898 326.12,269.145 
 327.49,272.939 328.86,273.246 330.23,274.552 331.601,272.948 332.971,276.536 334.341,278.527 335.712,281.907 337.082,285.383 338.452,287.513 339.822,289.34 
 341.193,290.632 342.563,271.906 343.933,273.192 345.303,267.921 346.674,271.7 348.044,260.801 349.414,262.587 350.785,257.966 352.155,261.437 353.525,251.594 
 354.895,251.355 356.266,239.758 357.636,239.655 359.006,226.934 360.376,223.522 361.747,219.

# Mean-variance Frontier

In [8]:
Rf    = 0.04                       #some inputs, here riskfree
mu    = [0.125; 0.105; 0.07]       #mean returns
stdd  = [0.129; 0.08; 0.1]         #std
Corr  = [ 1   0.33   0.45;         #correlation matrix
          0.33  1    0.05;
          0.45  0.05  1];
Sigma  = (stdd*stdd').*Corr        #covariance matrix
N      = length(mu)                #number of assets

mu_p = linspace(0,0.15,151)        #average returns to calculate min(Std(R_p)) at

151-element LinSpace{Float64}:
 0.0,0.001,0.002,0.003,0.004,0.005,0.006,…,0.145,0.146,0.147,0.148,0.149,0.15

In [9]:
ettor   = ones(N)                  #ettor means ones in Swedish: this a vector of those
Sigma_1 = inv(Sigma)               #mean-variance arithmetic
A       = (ettor'Sigma_1*mu)[1]          # [1] transforms from 1x1 array to scalar
B       = (mu'Sigma_1*mu)[1]
C       = (ettor'Sigma_1*ettor)[1]
D       = B*C - A^2
K       = length(mu_p)

Var_p = fill(NaN,K)                        #to store results in
for i = 1:K                                #loop over mean returns in Std x  mean figure
  g        = ( B*(Sigma_1*ettor) - A*(Sigma_1*mu) )/D
  h        = ( C*(Sigma_1*mu)  - A*(Sigma_1*ettor) )/D
  w_pi     = g + h.*mu_p[i]                        #portfolio weights at mean mu_p[i]
  Var_p[i] = (w_pi'Sigma*w_pi)[1]                  #variance of portfolio with mean mu_p[i]
end

SR_T    = sqrt.((mu-Rf)'Sigma_1*(mu-Rf))          #Sharpe ratio of Tangency portfolio
CML_std = sqrt.(((mu_p-Rf)./SR_T).^2)             #std according to CLM

151-element Array{Float64,1}:
 0.0437051
 0.0426125
 0.0415199
 0.0404273
 0.0393346
 0.038242 
 0.0371494
 0.0360567
 0.0349641
 0.0338715
 0.0327789
 0.0316862
 0.0305936
 ⋮        
 0.10817  
 0.109263 
 0.110355 
 0.111448 
 0.112541 
 0.113633 
 0.114726 
 0.115819 
 0.116911 
 0.118004 
 0.119097 
 0.120189 

In [10]:
plot1 = plot(sqrt.(Var_p),mu_p,color=:red,label="risky assets")
plot!(CML_std,mu_p,color=:blue,label="risky & riskfree ")
title!("Mean-variance frontier")
xlabel!("Std")
ylabel!("mean")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 0.125 
 
 
 Mean-variance frontier 
 
 
 Std 
 
 
 mean 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,369.674 590.747,367.365 585.436,365.057 580.132,362.748 574.833,360.44 569.54,358.131 564.254,355.822 558.975,353.514 553.702,351.205 548.436,348.897 
 543.178,346.588 537.927,344.279 532.683,341.971 527.448,339.662 522.221,337.353 517.003,335.045 511.793,332.736 506.593,330.428 501.402,328.119 496.221,325.81 
 491.05,323.502 485.89,321.193 480.74,318.885 475.602,316.576 470.476,314.267 465.362,311.959 460.261,309.65 455.173,307.341 450.099,305.033 445.039,302.724 
 439.994,300.416 434.965,298.107 429.952,295.798 424.955,293.49 419.976,291.181 415.015,288.872 410.073,286.564 405.152,284.255 400.25,281.947 395.371,279.638 
 390.514,277.329 385.681,275.021 380.872,272.712 376.09,270.404 371.334,268.095 366.607,265.786 361.909,263.478 357.243,261.169 352.609,258.86 348.009,256.552 
 343.446,254.243 338.92,251.935 334.433,249.626 329.989,247.317 325.587,245.009 321.232,242.7 316.925,240.392 312.669,238.083 308.467,235.774 304.321,233.466 
 300.234,231.157 296.209,228.848 292.25,226.54 288.36,224.231 284.543,221.923 280.802,219.614 277.142,217.305 273.567,214.997 270.082,212.688 266.69,210.379 
 263.396,208.071 260.207,205.762 257.126,203.454 254.159,201.145 251.311,198.836 248.589,196.528 245.997,194.219 243.541,191.911 241.227,189.602 239.06,187.293 
 237.046,184.985 235.191,182.676 233.499,180.367 231.975,178.059 230.625,175.75 229.452,173.442 228.46,171.133 227.652,168.824 227.032,166.516 226.601,164.207 
 226.36,161.899 226.311,159.59 226.454,157.281 226.788,154.973 227.313,152.664 228.025,150.355 228.923,148.047 230.004,145.738 231.265,143.43 232.701,141.121 
 234.307,138.812 236.08,136.504 238.014,134.195 240.103,131.886 242.343,129.578 244.727,127.269 247.25,124.961 249.907,122.652 252.692,120.343 255.598,118.035 
 258.622,115.726 261.757,113.418 264.998,111.109 268.34,108.8 271.779,106.492 275.309,104.183 278.926,101.874 282.626,99.5658 286.404,97.2572 290.258,94.9486 
 294.182,92.64 298.173,90.3314 302.229,88.0227 306.345,85.7141 310.52,83.4055 314.749,81.0969 319.03,78.7883 323.361,76.4797 327.739,74.171 332.162,71.8624 
 336.627,69.5538 341.133,67.2452 345.678,64.9366 350.259,62.628 354.876,60.3193 359.526,58.0107 364.208,55.7021 368.92,53.3935 373.661,51.0849 378.43,48.7762 
 383.226,46.4676 388.046,44.159 392.891,41.8504 397.76,39.5418 402.65,37.2332 407.561,34.9245 412.493,32.6159 417.444,30.3073 422.414,27.9987 427.402,25.6901 
 432.406,23.3815 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 174.844,369.674 172.064,367.365 169.285,365.057 166.505,362.748 163.726,360.44 160.946,358.131 158.166,355.822 155.387,353.514 152.607,351.205 149.827,348.897 
 147.048,346.588 144.268,344.279 141.489,341.971 138.709,339.662 135.929,337.353 133.15,335.045 130.37,332.736 127.591,330.428 124.811,328.119 122.031,325.81 
 119.252,323.502 116.472,321.193 113.692,318.885 110.913,316.576 108.133,314.267 105.354,311.959 102.574,309.65 99.7943,307.341 97.0147,305.033 94.2351,302.724 
 91.4554,300.416 88.6758,298.107 85.8962,295.798 83.1166,293.49 80.337,291.181 77.5573,288.872 74.7777,286.564 71.9981,284.255 69.2185,281.947 66.4389,279.638 
 63.6592,277.329 66.4389,275.021 69.2185,272.712 71.9981,270.404 74.7777,268.095 77.5573,265.786 80.337,263.478 83.1166,261.169 85.8962,258.86 88.6758,256.552 
 91.4554,254.243 94.2351,251.935 97.0147,249.626 99.7943,247.317 102.574,245.009 105.354,242.7 108.133,240.392 110.913,238.083 113.692,235.774 116.472,233.466 
 119.252,231.157 122.031,228.848 124.811,226.54 127.591,224.231 130.37,221.923 133.15,219.614 135.

# (extra) Mean-variance Frontier without Short Selling

In [11]:
using MathProgBase, Ipopt  

In [12]:
function MeanVarNoSSPs(mu,Sigma,mu_p)
#MeanVarNoSSPs     Calculate mean variance frontier when no short sales are allowed
#
#  Usage:  (Var_p,w_p) = MeanVarNoSSPs(mu,Sigma,mu_p)
#
#  Input:     mu        Nx1, vector of mean returns
#             Sigma     NxN, covariance matrix of returns, can contain riskfree assets
#             mu_p      Kx1, mean returns to calculate results for
#
#
#  Output:    Var_p     Kx1, variance of mean-variance portfolio (risky only) with mean mu_p
#             w_p       KxN, portfolio weights of       ""
#
#
#
#  Notice:  (a) uses quadprog from MathProgBase
#           (b) with a riskfree asset, put zeros row j and column j in Sigma
#------------------------------------------------------------------------------

  K = length(mu_p)                #MV with no short-sales, numerical minimization
  N = length(mu)

  vv = find((mu_p .>= minimum(mu)) & (mu_p .<= maximum(mu)))  #solve only if feasible

  lb   = zeros(N,1)              #w >= 0
  ub   = ones(N,1)               #w <= 1
  Aeq  = [ones(1,N);mu']         #1'w=1, mu'w = mu_p

  w_p   = fill(NaN,(K,N))
  Var_p = fill(NaN,K)                       #to store results in
  for i = vv
    beq  = [1;mu_p[i]]
    Sol = quadprog(zeros(N),Sigma,Aeq,'=',beq,zeros(N),ones(N),IpoptSolver(print_level=1))
    w_i = Sol.sol
    if Sol.status == :Optimal
      w_p[i,:] = w_i'
      Var_p[i] = (w_i'*Sigma*w_i)[1]
    end
  end

  return  Var_p,w_p

end

MeanVarNoSSPs (generic function with 1 method)

In [13]:
Var_no_ss, = MeanVarNoSSPs(mu,Sigma,mu_p)   #calculating MV without short selling

([NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN  …  NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN],
[NaN NaN NaN; NaN NaN NaN; … ; NaN NaN NaN; NaN NaN NaN])

In [14]:
plot1 = plot(sqrt.(Var_p),mu_p,color=:red,label="risky assets")
plot!(CML_std,mu_p,color=:blue,label="risky & riskfree ")
plot!(sqrt.(Var_no_ss),mu_p,color=:green,label="risky, no ss")
title!("Mean-variance frontier")
xlabel!("Std")
ylabel!("mean")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 0.125 
 
 
 Mean-variance frontier 
 
 
 Std 
 
 
 mean 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,369.674 590.747,367.365 585.436,365.057 580.132,362.748 574.833,360.44 569.54,358.131 564.254,355.822 558.975,353.514 553.702,351.205 548.436,348.897 
 543.178,346.588 537.927,344.279 532.683,341.971 527.448,339.662 522.221,337.353 517.003,335.045 511.793,332.736 506.593,330.428 501.402,328.119 496.221,325.81 
 491.05,323.502 485.89,321.193 480.74,318.885 475.602,316.576 470.476,314.267 465.362,311.959 460.261,309.65 455.173,307.341 450.099,305.033 445.039,302.724 
 439.994,300.416 434.965,298.107 429.952,295.798 424.955,293.49 419.976,291.181 415.015,288.872 410.073,286.564 405.152,284.255 400.25,281.947 395.371,279.638 
 390.514,277.329 385.681,275.021 380.872,272.712 376.09,270.404 371.334,268.095 366.607,265.786 361.909,263.478 357.243,261.169 352.609,258.86 348.009,256.552 
 343.446,254.243 338.92,251.935 334.433,249.626 329.989,247.317 325.587,245.009 321.232,242.7 316.925,240.392 312.669,238.083 308.467,235.774 304.321,233.466 
 300.234,231.157 296.209,228.848 292.25,226.54 288.36,224.231 284.543,221.923 280.802,219.614 277.142,217.305 273.567,214.997 270.082,212.688 266.69,210.379 
 263.396,208.071 260.207,205.762 257.126,203.454 254.159,201.145 251.311,198.836 248.589,196.528 245.997,194.219 243.541,191.911 241.227,189.602 239.06,187.293 
 237.046,184.985 235.191,182.676 233.499,180.367 231.975,178.059 230.625,175.75 229.452,173.442 228.46,171.133 227.652,168.824 227.032,166.516 226.601,164.207 
 226.36,161.899 226.311,159.59 226.454,157.281 226.788,154.973 227.313,152.664 228.025,150.355 228.923,148.047 230.004,145.738 231.265,143.43 232.701,141.121 
 234.307,138.812 236.08,136.504 238.014,134.195 240.103,131.886 242.343,129.578 244.727,127.269 247.25,124.961 249.907,122.652 252.692,120.343 255.598,118.035 
 258.622,115.726 261.757,113.418 264.998,111.109 268.34,108.8 271.779,106.492 275.309,104.183 278.926,101.874 282.626,99.5658 286.404,97.2572 290.258,94.9486 
 294.182,92.64 298.173,90.3314 302.229,88.0227 306.345,85.7141 310.52,83.4055 314.749,81.0969 319.03,78.7883 323.361,76.4797 327.739,74.171 332.162,71.8624 
 336.627,69.5538 341.133,67.2452 345.678,64.9366 350.259,62.628 354.876,60.3193 359.526,58.0107 364.208,55.7021 368.92,53.3935 373.661,51.0849 378.43,48.7762 
 383.226,46.4676 388.046,44.159 392.891,41.8504 397.76,39.5418 402.65,37.2332 407.561,34.9245 412.493,32.6159 417.444,30.3073 422.414,27.9987 427.402,25.6901 
 432.406,23.3815 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 174.844,369.674 172.064,367.365 169.285,365.057 166.505,362.748 163.726,360.44 160.946,358.131 158.166,355.822 155.387,353.514 152.607,351.205 149.827,348.897 
 147.048,346.588 144.268,344.279 141.489,341.971 138.709,339.662 135.929,337.353 133.15,335.045 130.37,332.736 127.591,330.428 124.811,328.119 122.031,325.81 
 119.252,323.502 116.472,321.193 113.692,318.885 110.913,316.576 108.133,314.267 105.354,311.959 102.574,309.65 99.7943,307.341 97.0147,305.033 94.2351,302.724 
 91.4554,300.416 88.6758,298.107 85.8962,295.798 83.1166,293.49 80.337,291.181 77.5573,288.872 74.7777,286.564 71.9981,284.255 69.2185,281.947 66.4389,279.638 
 63.6592,277.329 66.4389,275.021 69.2185,272.712 71.9981,270.404 74.7777,268.095 77.5573,265.786 80.337,263.478 83.1166,261.169 85.8962,258.86 88.6758,256.552 
 91.4554,254.243 94.2351,251.935 97.0147,249.626 99.7943,247.317 102.574,245.009 105.354,242.7 108.133,240.392 110.913,238.083 113.692,235.774 116.472,233.466 
 119.252,231.157 122.031,228.848 124.811,226.54 127.591,224.231 130.37,221.923 133.15,219.614 135.

# Options

### Black-Scholes Option Price

In [15]:
function Stdn_cdfPs(a)
#Stdn_cdfPs    Calculates Pr(X<=a) for X which is N(0,1)
  cdf = 0.5*SpecialFunctions.erfc.(-a/sqrt(2))
  return cdf
end

function OptionBlackSPs(S,X,T,r,sigma)
#Calculates Black-Scholes european call option price
  d1 = ( log.(S./X) + (r+1/2*sigma.^2)*T ) ./ (sigma*sqrt(T))
  d2 = d1 - sigma*sqrt(T)
  c  = S.*Stdn_cdfPs(d1) - X.*exp(-r*T).*Stdn_cdfPs(d2)
  return c
end

OptionBlackSPs (generic function with 1 method)

In [16]:
sigma = 0.4
c1 = OptionBlackSPs(10,10,0.5,0.1,sigma)
printlnPs("\n","call price according to Black-Scholes: ",c1)

X = linspace(7,13,51)
c = OptionBlackSPs(10,X,0.5,0.1,sigma)


call price according to Black-Scholes:      1.358


51-element Array{Float64,1}:
 3.41867 
 3.31625 
 3.21509 
 3.11527 
 3.01685 
 2.9199  
 2.82449 
 2.73066 
 2.63849 
 2.54802 
 2.4593  
 2.37238 
 2.28729 
 ⋮       
 0.712569
 0.678616
 0.646071
 0.614891
 0.585035
 0.55646 
 0.529124
 0.502986
 0.478005
 0.454139
 0.431351
 0.409599

In [17]:
plot1 = plot(X,c,color=:red,legend=false)
title!("Black-Scholes call option price")
xlabel!("strike price")
ylabel!("option price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 7 
 
 
 8 
 
 
 9 
 
 
 10 
 
 
 11 
 
 
 12 
 
 
 13 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 Black-Scholes call option price 
 
 
 strike price 
 
 
 option price 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 28.1037,23.3815 39.4629,35.1682 50.822,46.8095 62.1812,58.2973 73.5404,69.6236 84.8996,80.7808 96.2588,91.7615 107.618,102.559 118.977,113.166 130.336,123.578 
 141.696,133.788 153.055,143.792 164.414,153.584 175.773,163.161 187.132,172.519 198.491,181.656 209.851,190.568 221.21,199.253 232.569,207.711 243.928,215.941 
 255.287,223.941 266.647,231.712 278.006,239.255 289.365,246.57 300.724,253.66 312.083,260.525 323.443,267.167 334.802,273.591 346.161,279.797 357.52,285.789 
 368.879,291.572 380.238,297.147 391.598,302.52 402.957,307.693 414.316,312.673 425.675,317.462 437.034,322.065 448.394,326.487 459.753,330.733 471.112,334.807 
 482.471,338.715 493.83,342.46 505.19,346.048 516.549,349.484 527.908,352.773 539.267,355.919 550.626,358.927 561.985,361.802 573.345,364.548 584.704,367.171 
 596.063,369.674 
 "/>

# Implied Volatility

In [18]:
                                #simple (crude) way to solve for implied vol
iv = Roots.fzero(sigma->OptionBlackSPs(10,10,0.5,0.1,sigma)-c1,[-1;1])
printlnPs("Implied volatility for theoretical case (should be the same as above): ",iv)

Implied volatility for theoretical case (should be the same as above):      0.400


In [19]:
#  LIFFE Bunds option data, trade date April 6, 1994
X = [                        #strike prices; Mx1 vector
      92.00;  94.00;  94.50;  95.00;  95.50;  96.00;  96.50;  97.00;
      97.50;  98.00;  98.50;  99.00;  99.50;  100.0;  100.5;  101.0;
     101.5;  102.0;  102.5;  103.0;  103.5 ];
C = [                        #call prices; Mx1 vector
      5.13;    3.25;    2.83;    2.40;    2.00;    1.64;    1.31;    1.02;
      0.770;   0.570;   0.400;   0.280;   0.190;   0.130;  0.0800;  0.0500;
      0.0400;  0.0300;  0.0200;  0.0100;  0.0100 ];
F = 97.05                #Forward price
m = 48/365               #time to expiry in years
r = 0.0                  #Interest rate: LIFFE=>no discounting
N = length(X)

21

In [20]:
iv = fill(NaN,N)
for i = 1:N
  iv[i] = Roots.fzero(sigma->OptionBlackSPs(exp(-m*r)*F,X[i],m,r,sigma)-C[i],[-1;1])
end
println("Strike and iv for data: ")
printmat([X iv])

Strike and iv for data: 
    92.000     0.094
    94.000     0.081
    94.500     0.081
    95.000     0.078
    95.500     0.075
    96.000     0.074
    96.500     0.072
    97.000     0.071
    97.500     0.070
    98.000     0.069
    98.500     0.068
    99.000     0.067
    99.500     0.067
   100.000     0.068
   100.500     0.067
   101.000     0.067
   101.500     0.070
   102.000     0.073
   102.500     0.074
   103.000     0.072
   103.500     0.077



In [21]:
plot1 = plot(X,iv,color=:red,legend=false)
title!("Implied volatility, Bunds options April 6, 1994")
xlabel!("strike price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 92 
 
 
 94 
 
 
 96 
 
 
 98 
 
 
 100 
 
 
 102 
 
 
 0.07 
 
 
 0.08 
 
 
 0.09 
 
 
 Implied volatility, Bunds options April 6, 1994 
 
 
 strike price